# StudentPerformancePredictionML
### This project understands how the student's performance (test scores) is affected by other variables such as Gender, Ethnicity, Parental level of education, Lunch and Test preparation course.

*******************************
*******************************
<br>

## Main Folder 👇

-----------------------
<br>

### setup.py 👇
<i><b>(/StudentPerformancePredictionML/setup.py)</b></i>

The `setup.py` is responsible for creating my machine learning project as a package


```python
# Import Libraries
from setuptools import find_packages,setup
from typing import List
```

```python
setup(
name='StudentPerformancePredictionML',   # Name of project
version='0.0.1',   # Version 
author='Ninad',    # Author name
author_email='ninad.karlekar@vsit.edu.in',  # Author email-id
packages=find_packages(),
install_requires=get_requirements('requirements.txt') # Install all packages in file
)
# When you use `find_packages()` in your setup.py file, it will search the current directory and its subdirectories for any packages that contain an __init__.py file,
```

```python
HYPEN_E_DOT='-e .'
def get_requirements(file_path:str)->List[str]:
    '''
    this function will return the list of requirements
    '''
    requirements=[]
    with open(file_path) as file_obj:
        # Read all packages written in requirements.txt
        requirements=file_obj.readlines()
        # replace /n with blank
        requirements=[req.replace("\n","") for req in requirements] 

        # Remove -e . from requirements.txt while reading
        if HYPEN_E_DOT in requirements:
            requirements.remove(HYPEN_E_DOT)   
    
    return requirements
```

-----------------------
<br>

### requirements.txt 👇
<i><b>(/StudentPerformancePredictionML/requirements.txt)</b></i>

`requirements.txt` file will have all packages that we will use while implimenting project

```python
pandas
numpy
seaborn
-e .                            # -e . will automatically trigger setup.py
```